In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np




import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold


C:\Users\Vivek\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

trai = pd.read_csv("train.csv")
tes = pd.read_csv("test.csv")
train = trai[['Unnamed: 0',
 'Months since Last Donation',
 'Number of Donations',
 'Months since First Donation',
 'Made Donation in March 2007']]
train.head()
test = tes[['Unnamed: 0',
 'Months since Last Donation',
 'Number of Donations',
 'Months since First Donation']]
test.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Months since First Donation
0,659,2,12,52
1,276,21,7,38
2,263,4,1,4
3,303,11,11,38
4,83,4,12,34


In [3]:
train.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Months since First Donation,Made Donation in March 2007
0,619,2,50,98,1
1,664,0,13,28,1
2,441,1,16,35,1
3,160,2,20,45,1
4,358,1,24,77,0


In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train.drop('Made Donation in March 2007',axis=1))
scaled_features = scaler.transform(train.drop('Made Donation in March 2007',axis=1))
df_feat = pd.DataFrame(scaled_features,columns=train.columns[:-1])
df_feat.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Months since First Donation
0,1.130125,-0.910739,7.772052,2.641824
1,1.337729,-1.155586,1.320468,-0.249946
2,0.308938,-1.033162,1.843569,0.039231
3,-0.987430,-0.910739,2.541038,0.452341
4,-0.073975,-1.033162,3.238506,1.774293


In [5]:
#Train-Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_features,train['Made Donation in March 2007'],
test_size=0.30)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=11)

knn.fit(X_train,y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=11, p=2,
           weights='uniform')
           

pred = knn.predict(test)
pred2 = knn.predict_proba(test)
pred2=pred2[:,1]
print(pred2)


[ 0.18181818  0.18181818  0.18181818  0.18181818  0.45454545  0.18181818
  0.18181818  0.18181818  0.18181818  0.18181818  0.18181818  0.18181818
  0.27272727  0.18181818  0.18181818  0.18181818  0.18181818  0.18181818
  0.27272727  0.18181818  0.18181818  0.36363636  0.18181818  0.09090909
  0.18181818  0.27272727  0.18181818  0.18181818  0.18181818  0.18181818
  0.27272727  0.18181818  0.18181818  0.18181818  0.18181818  0.18181818
  0.18181818  0.18181818  0.18181818  0.18181818  0.09090909  0.18181818
  0.18181818  0.18181818  0.18181818  0.09090909  0.18181818  0.18181818
  0.18181818  0.18181818  0.27272727  0.18181818  0.18181818  0.18181818
  0.18181818  0.18181818  0.45454545  0.18181818  0.36363636  0.36363636
  0.18181818  0.27272727  0.36363636  0.18181818  0.27272727  0.18181818
  0.18181818  0.27272727  0.27272727  0.18181818  0.27272727  0.18181818
  0.          0.          0.18181818  0.18181818  0.18181818  0.18181818
  0.18181818  0.18181818  0.18181818  0.18181818  0

In [7]:
submission=pd.DataFrame({"":test['Unnamed: 0'],"Made Donation in March 2007":pred2})
submission.to_csv("MajhaKnn211.csv",index=False)